# 車牌辨識系統

<img src="https://raw.githubusercontent.com/j82887/Artificial-Intelligence/master/00_Image/19_Implementation%20-%2004_Car_License_Plate.png" align="left" width="200">

In [ ]:
import cv2
import matplotlib.pyplot as plt
img_path = 'carPlate/resize001.bmp'

img = cv2.imread(img_path)
detector = cv2.CascadeClassifier("License_Plate_Haar_cascade.xml")
signs = detector.detectMultiScale(img, minSize = (76, 20), scaleFactor = 1.1, minNeighbors=10)

if len(signs) > 0:
    for (sx, sy, sw, sh) in signs:
        crop_img = img[sy:sy+sh, sx:sx+sw]
        gray_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV)
                              
        save_predict_name = []
        contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            (x, y, w, h) = cv2.boundingRect(contours[i])
            if w>5 and 39>h>29:
                number = binary_img[y:y+h, x:x+w]
                pad_number = cv2.copyMakeBorder(number, 13, 13, 10, 10, borderType=cv2.BORDER_CONSTANT)
                re_number = cv2.resize(pad_number, (28, 28), interpolation=cv2.INTER_CUBIC)
        
                intput_number = np.expand_dims(np.expand_dims(re_number.astype('float32')/255,-1),0)
                predict = cnn.predict(intput_number)
                predict_name = label_name[np.argmax(predict)]
                save_predict_name.append(predict_name)
        
                #plt.imshow(re_number, cmap='gray')
                #plt.show()

        # 顛倒list
        name = save_predict_name[::-1]
        print("".join(name))
else:
    print('沒有辨識到車牌!')